In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import shapiro
from statsmodels.formula.api import ols


In [2]:
CURRENT_DIR = Path.cwd()
PARENT_DIR = CURRENT_DIR.parent
DATA_DIR = PARENT_DIR / "T04_preprocess"
print(CURRENT_DIR)
print(DATA_DIR)

c:\Users\admin\Coding\research\weld-ml\src\P02_data\T05_EA
c:\Users\admin\Coding\research\weld-ml\src\P02_data\T04_preprocess


In [3]:
filepath = os.path.join(DATA_DIR, "S02/S02_residual_stress_imputed_20260104_114536.xlsx")
dfm = pd.read_excel(filepath)
dfm

,idx_excel_post,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,idx_excel_pre,sigma_x_pre,FWHM_pre,diff_sigma_x
0,4,AA5052,1,1,1400,60,10,13.0,2.55,2,-15.0,2.50,28.0
1,25,AA5052,2,1,1400,60,15,16.0,2.51,9,2.0,2.47,14.0
2,46,AA5052,3,1,1400,60,20,19.0,2.47,16,9.0,2.48,10.0
3,67,AA5052,4,1,1400,70,10,20.0,2.45,23,10.0,2.48,10.0
4,88,AA5052,5,1,1400,70,15,6.0,2.47,30,0.0,2.49,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1050,Center,50,7,1600,70,15,2.0,2.45,0,0.0,0.00,2.0
1130,1071,Center,51,7,1600,70,20,2.0,2.45,0,0.0,0.00,2.0
1131,1092,Center,52,7,1600,80,10,5.0,2.54,0,0.0,0.00,5.0
1132,1113,Center,53,7,1600,80,15,1.0,2.41,0,0.0,0.00,1.0


In [4]:
# Standardize columnes R, W, D

for col in ["R", "W", "D"]:
    mean = dfm[col].mean()
    std = dfm[col].std()
    dfm[col] = (dfm[col] - mean) / std
dfm

,idx_excel_post,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,idx_excel_pre,sigma_x_pre,FWHM_pre,diff_sigma_x
0,4,AA5052,1,1,-1.224205,-1.224205,-1.224205,13.0,2.55,2,-15.0,2.50,28.0
1,25,AA5052,2,1,-1.224205,-1.224205,0.000000,16.0,2.51,9,2.0,2.47,14.0
2,46,AA5052,3,1,-1.224205,-1.224205,1.224205,19.0,2.47,16,9.0,2.48,10.0
3,67,AA5052,4,1,-1.224205,0.000000,-1.224205,20.0,2.45,23,10.0,2.48,10.0
4,88,AA5052,5,1,-1.224205,0.000000,0.000000,6.0,2.47,30,0.0,2.49,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1050,Center,50,7,1.224205,0.000000,0.000000,2.0,2.45,0,0.0,0.00,2.0
1130,1071,Center,51,7,1.224205,0.000000,1.224205,2.0,2.45,0,0.0,0.00,2.0
1131,1092,Center,52,7,1.224205,1.224205,-1.224205,5.0,2.54,0,0.0,0.00,5.0
1132,1113,Center,53,7,1.224205,1.224205,0.000000,1.0,2.41,0,0.0,0.00,1.0


## Understanding the ANOVA table columns:

- `df`: Degrees of freedom - the number of independent pieces of information for each source
- `sum_sq`: Sum of squares - the total variation attributed to each factor
- `mean_sq`: Mean square - sum of squares divided by degrees of freedom (variance estimate)
- `F`: F-statistic - the ratio of factor variance to residual variance
- `PR(>F)`: P-value - probability of seeing this F-statistic by chance alone

## What is Type

- typ=1 (Type I): sequential SS, each term tested in the order it appears in the formula.
- typ=2, each term (factor or covariate) is tested after all other main effects but ignoring higher‑order interactions that include it.
  This is usually recommended when the model is reasonably balanced and you want tests that respect the marginality principle (main effects evaluated in the presence of other main effects, but not “penalized” by interactions).
- typ=3 (Type III): each term tested after all other terms including interactions; often used in software like SPSS, especially for unbalanced designs, but interpretation of main effects with strong interactions can be tricky.


In [5]:
# Filter the DataFrame for specific sections
filt = (dfm["section"].isin(["AA5052", "AA6061", "Center"])) 
dfa = dfm[filt]

# Define the formula for the model
formula = "diff_sigma_x ~ R + W + D + section"
# formula = "diff_sigma_x ~ C(R) + C(W) + C(D) + C(section)"

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

,df,sum_sq,mean_sq,F,PR(>F)
section,2.0,2.998238e+05,149911.913119,136.490649,8.212843e-54
W,1.0,7.278001e+03,7278.001131,6.626419,1.017440e-02
R,1.0,7.722643e+02,772.264342,0.703125,4.019132e-01
D,1.0,2.566683e+01,25.666829,0.023369,8.785289e-01
Residual,1128.0,1.238917e+06,1098.331012,NaN,NaN


In [6]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           diff_sigma_x   R-squared:                       0.199
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     56.07
Date:                Sun, 11 Jan 2026   Prob (F-statistic):           3.89e-52
Time:                        19:22:17   Log-Likelihood:                -5575.9
No. Observations:                1134   AIC:                         1.116e+04
Df Residuals:                    1128   BIC:                         1.119e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            24.0616      1.705     14.116      0.000      20.717      27.406
section[T.AA6061]   -39.0364      2.411    -16.193      0.000     -43.766     -34.307
section[T.Center]   -12.6700      2.411     -5.256      0.000     -17.400      -7.940
R                     0.8256      0.985      0.839      0.402      -1.106       2.757
W                     2.5345      0.985      2.574      0.010       0.603       4.466
D                     0.1505      0.985      0.153      0.879      -1.781       2.082
==============================================================================
Omnibus:                      102.612   Durbin-Watson:                   1.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              624.680
Skew:                           0.080   Prob(JB):                    2.25e-136
Kurtosis:                       6.633   Cond. No.                         3.73
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA5052"]) 
# filt = dfm["section"].isin(["AA5052"]) & (dfm["location"] == 1)
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
# t_main = ["R", "W", "D"]
t_main = ["W", "R", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_single = ["I(R)", "I(W)", "I(D)"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    # + t_interact_2
    # + t_interact_3
    # + t_single
    # + t_quad
    # + t_quad_pair
    # + t_triple_1
    # + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")
display(anova_results)

coefficients = model.params
print(coefficients)

diff_sigma_x ~ W + R + D


,df,sum_sq,mean_sq,F,PR(>F)
W,1.0,93.119462,93.119462,0.383574,0.536074
R,1.0,2571.379417,2571.379417,10.591922,0.001239
D,1.0,445.532194,445.532194,1.835218,0.176330
Residual,374.0,90795.223634,242.767978,NaN,NaN


Intercept    24.061634
W             0.496553
R             2.609328
D            -1.086139
dtype: float64


In [8]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           diff_sigma_x   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     4.270
Date:                Sun, 11 Jan 2026   Prob (F-statistic):            0.00555
Time:                        19:22:17   Log-Likelihood:                -1572.4
No. Observations:                 378   AIC:                             3153.
Df Residuals:                     374   BIC:                             3168.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.0616      0.801     30.024      0.000      22.486      25.637
W              0.4966      0.802      0.619      0.536      -1.080       2.073
R              2.6093      0.802      3.255      0.001       1.033       4.186
D             -1.0861      0.802     -1.355      0.176      -2.663       0.490
==============================================================================
Omnibus:                        0.070   Durbin-Watson:                   1.638
Prob(Omnibus):                  0.966   Jarque-Bera (JB):                0.129
Skew:                          -0.030   Prob(JB):                        0.938
Kurtosis:                       2.933   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["AA6061"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
W,1.0,1.556504e+04,15565.041884,5.195087,0.023234
R:W:D,1.0,8.424503e+03,8424.503270,2.811815,0.094438
R:D,1.0,8.395465e+03,8395.464633,2.802123,0.095006
I(R ** 2 * W),1.0,7.233430e+03,7233.429683,2.414275,0.121110
I(D ** 2 * R),1.0,4.193085e+03,4193.085474,1.399511,0.237584
I(W ** 2),1.0,3.093305e+03,3093.305276,1.032441,0.310267
I(R ** 2),1.0,1.850468e+03,1850.468148,0.617624,0.432447
I(D ** 2),1.0,1.836991e+03,1836.991357,0.613126,0.434127
W:D,1.0,1.676792e+03,1676.791787,0.559657,0.454886
I(R * W),1.0,1.477120e+03,1477.119940,0.493013,0.483039


In [10]:
# Filter the DataFrame for specific sections
filt = dfm["section"].isin(["Center"])
dfa = dfm[filt]

# Define the formula for the model
base = "diff_sigma_x ~ "
t_main = ["R", "W", "D"]
t_interact_2 = ["R:W", "R:D", "W:D"]
t_interact_3 = ["R:W:D"]
t_quad = ["I(R**2)", "I(W**2)", "I(D**2)"]
t_quad_pair = ["I(R*W)", "I(R*D)", "I(W*D)"]
t_triple_1 = [
    "I(R**2*W)",
    "I(R**2*D)",
    "I(W**2*R)",
    "I(W**2*D)",
    "I(D**2*R)",
    "I(D**2*W)",
]
t_triple_all = ["I(R*W*D)"]

formula = base + " + ".join(
    t_main
    + t_interact_2
    + t_interact_3
    + t_quad
    + t_quad_pair
    + t_triple_1
    + t_triple_all
)
print(formula)

# Fit the model
model = ols(formula, data=dfa).fit()

# Perform ANOVA
anova_results = sm.stats.anova_lm(model)

# Print the ANOVA results
anova_results.sort_values(by="PR(>F)")

diff_sigma_x ~ R + W + D + R:W + R:D + W:D + R:W:D + I(R**2) + I(W**2) + I(D**2) + I(R*W) + I(R*D) + I(W*D) + I(R**2*W) + I(R**2*D) + I(W**2*R) + I(W**2*D) + I(D**2*R) + I(D**2*W) + I(R*W*D)


,df,sum_sq,mean_sq,F,PR(>F)
R,1.0,525.777778,525.777778,11.801518,0.000661
R:W:D,1.0,180.035714,180.035714,4.041051,0.045150
W,1.0,178.317462,178.317462,4.002483,0.046182
I(R ** 2),1.0,78.726966,78.726966,1.767092,0.184582
I(R ** 2 * W),1.0,76.700777,76.700777,1.721613,0.190319
I(W ** 2 * D),1.0,57.410146,57.410146,1.288618,0.257056
R:W,1.0,52.595238,52.595238,1.180544,0.277970
I(D ** 2),1.0,50.912696,50.912696,1.142778,0.285780
D,1.0,49.710714,49.710714,1.115798,0.291531
I(D ** 2 * R),1.0,36.847127,36.847127,0.827064,0.363730


In [11]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           diff_sigma_x   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     1.903
Date:                Sun, 11 Jan 2026   Prob (F-statistic):             0.0191
Time:                        19:22:17   Log-Likelihood:                -1245.2
No. Observations:                 378   AIC:                             2524.
Df Residuals:                     361   BIC:                             2591.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        10.2165      0.908     11.248      0.000       8.430      12.003
R                -0.8136      0.768     -1.059      0.290      -2.324       0.697
W                 1.6568      0.768      2.157      0.032       0.147       3.167
D                 0.0254      0.768      0.033      0.974      -1.485       1.536
R:W               0.1867      0.172      1.087      0.278      -0.151       0.525
R:D               0.0298      0.172      0.173      0.862      -0.308       0.368
W:D              -0.1528      0.172     -0.889      0.375      -0.491       0.185
R:W:D            -0.3455      0.172     -2.010      0.045      -0.684      -0.008
I(R ** 2)         0.6460      0.486      1.329      0.185      -0.310       1.602
I(W ** 2)         0.0106      0.486      0.022      0.983      -0.945       0.966
I(D ** 2)         0.5195      0.486      1.069      0.286      -0.436       1.475
I(R * W)          0.1867      0.172      1.087      0.278      -0.151       0.525
I(R * D)          0.0298      0.172      0.173      0.862      -0.308       0.368
I(W * D)         -0.1528      0.172     -0.889      0.375      -0.491       0.185
I(R ** 2 * W)    -0.6358      0.486     -1.308      0.192      -1.592       0.320
I(R ** 2 * D)    -0.2299      0.486     -0.473      0.637      -1.186       0.726
I(W ** 2 * R)     0.0649      0.486      0.133      0.894      -0.891       1.021
I(W ** 2 * D)     0.5675      0.486      1.167      0.244      -0.389       1.524
I(D ** 2 * R)    -0.4315      0.486     -0.888      0.375      -1.388       0.525
I(D ** 2 * W)    -0.3348      0.486     -0.689      0.491      -1.291       0.621
I(R * W * D)     -0.3455      0.172     -2.010      0.045      -0.684      -0.008
==============================================================================
Omnibus:                        6.145   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.046   Jarque-Bera (JB):                4.697
Skew:                           0.160   Prob(JB):                       0.0955
Kurtosis:                       2.557   Cond. No.                     4.13e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.72e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""